## Exporting CSVs from the FairNow APIs

#### This notebook demonstrates how to generate CVSs from the FairNow APIs. The current supported outputs are 1. Severity/Probability Export, 2. Application Inventory Export, and 3. Compliance Export.

### Prerequisites

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

In [9]:
from getpass import getpass
import httpx
from httpx_auth import OAuth2ClientCredentials

client_id = "2726p9mj2ghit00n23tkjatp84" # Replace with your Client ID
client_secret = getpass("Client Secret")
fairnow_token_endpoint = "https://auth.fairnow.dev/oauth2/token"

auth = OAuth2ClientCredentials(
    token_url=fairnow_token_endpoint,
    client_id=client_id,
    client_secret=client_secret,
)

fairnow_base_url = "https://api.fairnow.dev/v2"
client = httpx.Client(base_url=fairnow_base_url, auth=auth)

## Create the df

In [16]:
import pandas as pd

application_route = "/applications"

apps_response = None
try:
    response = client.get(application_route, timeout=None)
    if response.status_code == 200:
        apps_response = response.json()
        print(f"Successfully retrieved applications")
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(e)

# Extract fields from response
extracted_data = []
for app in apps_response['applications']:
    app_id = app['application_id']
    app_name = app['application_name']
    risk_assessment = app.get('risk_assessment', {})
    framework_items = risk_assessment.get('framework_assessment_items', []) or []
    for framework in framework_items:
        extracted_data.append({
            'application_id': app_id,
            'application_name': app_name,
            'framework_id': framework.get('framework_id', ''),
            'framework_name': framework.get('framework_name', ''),
        })

# Convert to DataFrame
apps_df = pd.DataFrame(extracted_data)
apps_df = apps_df.drop_duplicates()
print(f"Extracted frameworks")
apps_df.head()

Successfully retrieved applications
Extracted frameworks


,application_id,application_name,framework_id,framework_name
0,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],colorado-sb-205-developer,Colorado AI Act - Developer
1,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],euact-gpai,EU AI Act - General Purpose AI
2,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],illinois-hb3773,Illinois HB 3773
3,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],maryland-hb-1202,Maryland HB1202 on Facial Recognition
4,05e0d38a-f18c-45ac-8eaf-216690f69e8d,Steve Test [renamed x2],utah-ai-policy-act,Utah AI Policy Act


## Frameworks

In [21]:
def get_framework_controls(framework_id):

    query_parameters = {"framework_id": framework_id}
    # response = client.get(f"/control/", params=query_parameters, timeout=None)

    application_route = f"/controls/framework/"

    framework_response = None
    
    try:
        response = client.get(application_route, params=query_parameters, timeout=None)
        if response.status_code == 200:
            framework_response = response.json()
            print(f"Successfully retrieved framework controls")
            return framework_response
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(e)
        return None

## Get Framework IDs and combine

In [22]:
framework_ids = apps_df['framework_id'].unique()

# Create a list to store all controls data
all_controls_data = []

# Fetch controls for each framework
for framework_id in framework_ids:
    print(f"Fetching controls for framework: {framework_id}")
    controls = get_framework_controls(framework_id)
    if controls:
        # Add framework_id to each control record
        for control in controls:
            control['framework_id'] = framework_id
        all_controls_data.extend(controls)

# Create a DataFrame from the controls data
controls_df = pd.DataFrame(all_controls_data)

# Merge with the original DataFrame
# This will add the control information to each framework row
merged_df = pd.merge(
    apps_df[['application_id', 'application_name', 'framework_id', 'framework_name']],
    controls_df[['framework_id', 'applications_count', 'applications_ready']],
    on='framework_id',
    how='left'
)

Fetching controls for framework: colorado-sb-205-developer
Successfully retrieved framework controls
Fetching controls for framework: euact-gpai
Successfully retrieved framework controls
Fetching controls for framework: illinois-hb3773
Successfully retrieved framework controls
Fetching controls for framework: maryland-hb-1202
Successfully retrieved framework controls
Fetching controls for framework: utah-ai-policy-act
Successfully retrieved framework controls
Fetching controls for framework: illinois-aivia
Successfully retrieved framework controls
Fetching controls for framework: nyc-ll-144
Successfully retrieved framework controls
Fetching controls for framework: colorado-sb-205-deployer
Successfully retrieved framework controls
Fetching controls for framework: euact-high-risk
Successfully retrieved framework controls
Fetching controls for framework: euact-low-risk
Successfully retrieved framework controls


### Export to TSV

In [23]:
# Export to TSV
output_file = 'frameworks_export.tsv'
merged_df.to_csv(output_file, sep='\t', index=False)
print(f"Data exported to {output_file}")

Data exported to application_risks.tsv
